In [2]:
import numpy as np
from numpy import nan_to_num, subtract, add, divide, multiply
from osgeo import gdal, gdalconst
from gdal import GetDriverByName

def ndvi(in_nir_band, in_colour_band, in_rows, in_cols, in_geotransform, out_tiff, data_type=gdal.GDT_Float32):

    """
    Performs an NDVI calculation given two input bands, as well as other information that can be retrieved from the
    original image.
    @param in_nir_band A GDAL band object representing the near-infrared image data.
    @type in_nir_band GDALRasterBand
    @param in_colour_band A GDAL band object representing the colour image data.
    @type: in_colour_band GDALRasterBand
    @param in_rows The number of rows in both input bands.
    @type: in_rows int
    @param in_cols The number of columns in both input bands.
    @type: in_cols int
    @param in_geotransform The geographic transformation to be applied to the output image.
    @type in_geotransform Tuple (as returned by GetGeoTransform())
    @param out_tiff Path to the desired output .tif file.
    @type: out_tiff String (should end in ".tif")
    @param data_type Data type of output image.  Valid values are gdal.UInt16 and gdal.Float32.  Default is
                      gdal.Float32
    @type data_type GDALDataType
    @return None
    """

    # Read the input bands as numpy arrays.
    np_nir = in_nir_band.ReadAsArray(0, 0, in_cols, in_rows)
    np_colour = in_colour_band.ReadAsArray(0, 0, in_cols, in_rows)

    # Convert the np arrays to 32-bit floating point to make sure division will occur properly.
    np_nir_as32 = np_nir.astype(np.float32)
    np_colour_as32 = np_colour.astype(np.float32)
    np.seterr(divide='ignore', invalid='ignore')

    # Calculate the NDVI formula.
    numerator = subtract(np_nir_as32, np_colour_as32)
    denominator = add(np_nir_as32, np_colour_as32)
    result = divide(numerator, denominator)
    
    return result
    # Remove any out-of-bounds areas
    result[result == -0] = -99

    # Initialize a geotiff driver.
    geotiff = GetDriverByName('GTiff')

    # If the desired output is an int16, map the domain [-1,1] to [0,255], create an int16 geotiff with one band and
    # write the contents of the int16 NDVI calculation to it.  Otherwise, create a float32 geotiff with one band and
    # write the contents of the float32 NDVI calculation to it.
    if data_type == gdal.GDT_UInt16:
        ndvi_int8 = multiply((result + 1), (2**7 - 1))
        output = geotiff.Create(out_tiff, in_cols, in_rows, 1, gdal.GDT_Byte)
        output_band = output.GetRasterBand(1)
        output_band.SetNoDataValue(-99)
        output_band.WriteArray(ndvi_int8)
    elif data_type == gdal.GDT_Float32:
        output = geotiff.Create(out_tiff, in_cols, in_rows, 1, gdal.GDT_Float32)
        output_band = output.GetRasterBand(1)
        output_band.SetNoDataValue(-99)
        output_band.WriteArray(result)
    else:
        raise ValueError('Invalid output data type.  Valid types are gdal.UInt16 or gdal.Float32.')

    # Set the geographic transformation as the input.
    output.SetGeoTransform(in_geotransform)

    return None



ImportError: libkea.so.1.4.7: cannot open shared object file: No such file or directory

In [9]:
np.__version__

'1.10.4'

In [24]:
lc08_nir = rasterio.open('./LC08_01_166_043_LC08_L1TP_166043_20180827_20180911_01_T1_LC08_L1TP_166043_20180827_20180911_01_T1_B4.TIF')
band_nir = lc08.read(1)
lc08_red = rasterio.open('./LC08_01_166_043_LC08_L1TP_166043_20180827_20180911_01_T1_LC08_L1TP_166043_20180827_20180911_01_T1_B3.TIF')
band_red = lc08_red.read(1)


In [25]:
band_red[np.nonzero(band_red)].shape, band_nir[np.nonzero(band_nir)].shape

((41393599,), (41393349,))

In [28]:
import gdal
from gdal import Open
from ndvi import ndvi


# Open NIR image and get its only band.
nir_tiff = Open(r'NIR_IMAGE.tif')
nir_band = nir_tiff.GetRasterBand(1)

# Open red image and get its only band.
red_tiff = Open(r'RED_IMAGE.tif')
red_band = red_tiff.GetRasterBand(1)


# Get the rows and cols from one of the images (both should always be the same)
rows, cols, geotransform = nir_tiff.RasterYSize, nir_tiff.RasterXSize, nir_tiff.GetGeoTransform()
#print(rows, cols,geotransform)

# Set an output for a 16-bit unsigned integer (0-255)
out_tiff_int16 = r'NDVI_INT16.tif'

# Set the output for a 32-bit floating point (-1 to 1)
out_tiff_float32 = r'NDVI_FLOAT32.tif'


#Run the function for unsigned 16-bit integer
#ndvi(nir_band, red_band, rows, cols, geotransform, out_tiff_int16, gdal.GDT_UInt16)

# # Run the function for 32-bit floating point
val = ndvi(nir_band, red_band, rows, cols, geotransform, out_tiff_float32, gdal.GDT_Float32)

print(val)

None


In [2]:
import rasterio
import numpy as np
import cupy as cp
from numba import jit

In [3]:
rasterio.plot()

AttributeError: module 'rasterio' has no attribute 'plot'

In [4]:
@jit()
def ndvi(band1, band2):
    np_nir_as32 = band1.astype(np.float32)
    np_colour_as32 = band2.astype(np.float32)

    np.seterr(divide='ignore', invalid='ignore')

    # Calculate the NDVI formula.
    numerator = np.subtract(np_nir_as32, np_colour_as32)
    denominator = np.add(np_nir_as32, np_colour_as32)
    result = np.divide(numerator, denominator)
    return result



In [6]:
nir = rasterio.open('./finally.TIF')
x, y, geo =nir.height, nir.width, nir.get_transform()
band1 = nir.read(1)

red = rasterio.open('./finally2.TIF')
band2 = nir.read(1)

In [7]:
nd = ndvi(band1, band2)

array([], shape=(0, 2339), dtype=float32)

In [33]:
nd[np.not_equal(np.nan)]

ValueError: invalid number of arguments

In [64]:
# Define spatial characteristics of output object (basically they are analog to the input)
kwargs = nir.meta

# Update kwargs (change in data type)
kwargs.update(
    dtype=rasterio.float32,
    count = 1)

# Let's see what is in there
print(kwargs)

with rasterio.open('data/ndvi.tif', 'w', **kwargs) as dst:
        dst.write_band(1, nd.astype(rasterio.float32))

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 7591, 'height': 7741, 'count': 1, 'crs': {'init': 'epsg:32638'}, 'transform': (482685.0, 30.0, 0.0, 2831415.0, 0.0, -30.0), 'affine': Affine(30.0, 0.0, 482685.0,
       0.0, -30.0, 2831415.0)}


/usr/local/lib/python3.6/dist-packages/rasterio/__init__.py:99: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


ValueError: NULL dataset

In [ ]:
nd.max(axis=1).max()

In [24]:
import osmnx as ox
import matplotlib.pyplot as plt
import rasterio
import numpy as np
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
from rasterio.mask import mask

In [27]:
#Now we need to initialize the ndvi with zeros before we do the calculations (this is numpy specific trick)

# def ndvi(band1, band2):
nir_tiff = rasterio.open('./finally.TIF')
band1 = nir_tiff.read(1)

red_tiff = rasterio.open('./finally2.TIF')
band2 = red_tiff.read(1)

nir = band1.astype(np.float32)
red = band2.astype(np.float32)

np.seterr(divide='ignore', invalid='ignore')

ndvi = np.empty(nir_tiff.shape, dtype=rasterio.float32)

check = np.logical_or( red > 0, nir > 0 )

ndvi = np.where (check,  (nir - red ) / ( nir + red ), -999 )
#     return ndvi


ndvi.shape

(2385, 2339)

In [60]:
nir_tiff.bounds

BoundingBox(left=644355.0, bottom=2700885.0, right=714525.0, top=2772435.0)

In [150]:
from shapely.geometry import mapping, LineString, MultiLineString, Polygon
minx, miny, maxx, maxy = 644355.0, 2700885.0, 714525.0, 2772435.0
num_tiles = 4

dx = (maxx - minx) / num_tiles
dy = (maxy - miny) / num_tiles
lines = []
for x in range(1, num_tiles+1):
        lines.append([(minx + x * dx), (miny + x * dy), (maxx - x * dx),(maxy - x * dy )])

lines, nir_tiff.bounds    
# for x in range(num_tiles + 1):
#     lines.append(LineString([(minx + x * dx, miny), (minx + x * dx, maxy)]))
# for y in range(num_tiles + 1):
#     lines.append(LineString([(minx, miny + y * dy), (maxx, miny + y * dy)]))

([[661897.5, 2718772.5, 696982.5, 2754547.5],
  [679440.0, 2736660.0, 679440.0, 2736660.0],
  [696982.5, 2754547.5, 661897.5, 2718772.5],
  [714525.0, 2772435.0, 644355.0, 2700885.0]],
 BoundingBox(left=644355.0, bottom=2700885.0, right=714525.0, top=2772435.0))

In [148]:
from shapely.geometry import Polygon
import numpy as np
xmin,ymin,xmax,ymax =  nir_tiff.bounds
width = 20000
height = 20000
rows = int(np.ceil((ymax-ymin) /  height))
cols = int(np.ceil((xmax-xmin) / width))
XleftOrigin = xmin
XrightOrigin = xmin + width
YtopOrigin = ymax
YbottomOrigin = ymax- height

polygons = []
for i in range(cols):
    Ytop = YtopOrigin
    Ybottom =YbottomOrigin
    for j in range(rows):
        polygons.append(Polygon([(XleftOrigin, Ytop), (XrightOrigin, Ytop), (XrightOrigin, Ybottom), (XleftOrigin, Ybottom)])) 
        Ytop = Ytop - height
        Ybottom = Ybottom - height
    XleftOrigin = XleftOrigin + width
    XrightOrigin = XrightOrigin + width
len(polygons)

16

In [159]:
import math
nd = 5578515
N = 4
n = int(math.sqrt(N))
d = nd / n
c = [i*d+d/2 for i in range(n)]
[[(x,y) for x in c] for y in c]


[[(1394628.75, 1394628.75), (4183886.25, 1394628.75)],
 [(1394628.75, 4183886.25), (4183886.25, 4183886.25)]]

In [157]:
 nir_tiff.height * nir_tiff.width

5578515

In [151]:
from shapely.geometry import box
xmin,ymin,xmax,ymax =  nir_tiff.bounds
b = box(xmin,ymin,xmax,ymax)

for i in lines:
    if not i.within(b):
        print(i.bounds)
nir_tiff.bounds

AttributeError: 'list' object has no attribute 'within'

In [69]:
def bounded_segments(lines, bounding_box, cut_segment=True):
    """
    Extract the bounded segments from a list of lines
    :param lines: a list of LineString
    :param bounding_box: the bounding coordinates in (minx, miny, maxx, maxy)
           or Polygon instance
    :return: a list of bounded segments
    """
    if isinstance(bounding_box, Polygon):
        bbox = bounding_box
    else:
        bbox = box(bounding_box[0], bounding_box[1],
                   bounding_box[2], bounding_box[3])
    segments = []
    for line in lines:
        if line.intersects(bbox):
            if cut_segment:
                segments.append(line.intersection(bbox))
            else:
                segments.append(line)
    return segments 

In [107]:
import math
minx, miny, maxx, maxy = nir_tiff.bounds
dx = 100
dy = 100

nx = int(math.ceil(abs(maxx - minx)/dx))
ny = int(math.ceil(abs(maxy - miny)/dy))

id_=0

for i in range(ny):
    for j in range(nx):
        id_+=1
        vertices = []
        parts = []
        vertices.append([min(minx+dx*j,maxx),max(maxy-dy*i,miny)])
        vertices.append([min(minx+dx*(j+1),maxx),max(maxy-dy*i,miny)])
        vertices.append([min(minx+dx*(j+1),maxx),max(maxy-dy*(i+1),miny)])
        vertices.append([min(minx+dx*j,maxx),max(maxy-dy*(i+1),miny)])
        parts.append(vertices)
        
parts

[[[714455.0, 2700935.0],
  [714525.0, 2700935.0],
  [714525.0, 2700885.0],
  [714455.0, 2700885.0]]]

In [85]:
from shapely.ops import triangulate
triangulate(nir_tiff.bounds)

AttributeError: 'BoundingBox' object has no attribute '_geom'

In [19]:
# Breaking down the tiff into sub-sections
minx, miny = 672631.507723, 2737061.5984
maxx, maxy = 690208.183352, 2728691.75286

bbox = box(minx, miny, maxx, maxy)

geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(32638))

geo = geo.to_crs(crs=nir_tiff.crs.data)

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]


coords = getFeatures(geo)

out_img, out_transform = mask(nir_tiff, shapes=coords, crop=True)

In [26]:
out_img.shape

(1, 280, 587)

In [37]:
#ox.plot_graph(ox.graph_from_place('Riyadh Province'))
city = ox.gdf_from_place('Riyadh, Saudi Arabia', which_result=1, buffer_dist=10000)
city
# city = ox.project_gdf(city)
# fig, ax = ox.plot_shape(city, figsize=(3,3))

bbox_east  bbox_north  bbox_south  bbox_west  \
0  46.875065   24.791969   24.471969  46.555065   

                                            geometry  \
0  POLYGON ((46.81382347948706 24.63081019575558,...   

                                   place_name  
0  Riyadh, Riyadh Region, 11131, Saudi Arabia

In [ ]:
Al Malqa, Al Aqiq, Hittin = 657449.451015 2746674.41104 666375.416619 2736628.58259

Al Yasmin, As Sahafah, Al Falah = 


In [8]:
ndvi.mean()

-0.054284677

In [24]:
import pandas as pd

In [25]:
landsat8 = pd.read_csv('./LANDSAT_8_C1_281138.csv', encoding = "ISO-8859-1", parse_dates=['Acquisition Date'])

In [26]:
landsat8

Landsat Product Identifier Landsat Scene Identifier  \
0    LC08_L1TP_166043_20180928_20180928_01_RT    LC81660432018271LGN00   
1    LC08_L1TP_165043_20180921_20180928_01_T1    LC81650432018264LGN00   
2    LC08_L1TP_166043_20180912_20180927_01_T1    LC81660432018255LGN00   
3    LC08_L1TP_165043_20180905_20180912_01_T1    LC81650432018248LGN00   
4    LC08_L1TP_166043_20180827_20180911_01_T1    LC81660432018239LGN00   
5    LC08_L1TP_165043_20180820_20180829_01_T1    LC81650432018232LGN01   
6    LC08_L1TP_166043_20180811_20180815_01_T1    LC81660432018223LGN00   
7    LC08_L1TP_165043_20180804_20180814_01_T1    LC81650432018216LGN00   
8    LC08_L1TP_166043_20180726_20180731_01_T1    LC81660432018207LGN00   
9    LC08_L1TP_165043_20180719_20180731_01_T1    LC81650432018200LGN00   
10   LC08_L1TP_166043_20180710_20180717_01_T1    LC81660432018191LGN00   
11   LC08_L1TP_165043_20180703_20180717_01_T1    LC81650432018184LGN00   
12   LC08_L1TP_166043_20180624_20180704_01_T1    LC81660432018175LGN00   
13   LC08_L1TP_165043_20180617_20180703_01_T1    LC81650432018168LGN00   
14   LC08_L1TP_166043_20180608_20180615_01_T1    LC81660432018159LGN00   
15   LC08_L1TP_165043_20180601_20180614_01_T1    LC81650432018152LGN00   
16   LC08_L1TP_166043_20180523_20180605_01_T1    LC81660432018143LGN00   
17   LC08_L1TP_165043_20180516_20180604_01_T1    LC81650432018136LGN00   
18   LC08_L1TP_166043_20180507_20180517_01_T1    LC81660432018127LGN00   
19   LC08_L1TP_165043_20180430_20180502_01_T1    LC81650432018120LGN00   
20   LC08_L1TP_166043_20180421_20180502_01_T1    LC81660432018111LGN00   
21   LC08_L1TP_165043_20180414_20180501_01_T1    LC81650432018104LGN00   
22   LC08_L1TP_166043_20180405_20180417_01_T1    LC81660432018095LGN00   
23   LC08_L1TP_165043_20180329_20180405_01_T1    LC81650432018088LGN00   
24   LC08_L1TP_166043_20180320_20180403_01_T1    LC81660432018079LGN00   
25   LC08_L1TP_165043_20180313_20180320_01_T1    LC81650432018072LGN00   
26   LC08_L1TP_166043_20180304_20180319_01_T1    LC81660432018063LGN00   
27   LC08_L1TP_165043_20180225_20180308_01_T1    LC81650432018056LGN00   
28   LC08_L1TP_166043_20180216_20180307_01_T1    LC81660432018047LGN00   
29   LC08_L1TP_165043_20180209_20180221_01_T1    LC81650432018040LGN00   
..                                        ...                      ...   
226  LC08_L1TP_165043_20131126_20170428_01_T1    LC81650432013330LGN01   
227  LC08_L1TP_165043_20131110_20180526_01_T1    LC81650432013314LGN02   
228  LC08_L1TP_166043_20131101_20170429_01_T1    LC81660432013305LGN01   
229  LC08_L1TP_165043_20131025_20170429_01_T1    LC81650432013298LGN01   
230  LC08_L1TP_166043_20131016_20170429_01_T1    LC81660432013289LGN01   
231  LC08_L1TP_165043_20131009_20170429_01_T1    LC81650432013282LGN01   
232  LC08_L1TP_165043_20130923_20170502_01_T1    LC81650432013266LGN01   
233  LC08_L1TP_166043_20130914_20180526_01_T1    LC81660432013257LGN02   
234  LC08_L1TP_165043_20130907_20170502_01_T1    LC81650432013250LGN01   
235  LC08_L1TP_166043_20130829_20170502_01_T1    LC81660432013241LGN01   
236  LC08_L1TP_165043_20130822_20170503_01_T1    LC81650432013234LGN02   
237  LC08_L1TP_166043_20130813_20180526_01_T1    LC81660432013225LGN02   
238  LC08_L1TP_165043_20130806_20170503_01_T1    LC81650432013218LGN01   
239  LC08_L1TP_166043_20130728_20170503_01_T1    LC81660432013209LGN01   
240  LC08_L1TP_165043_20130721_20170503_01_T1    LC81650432013202LGN01   
241  LC08_L1TP_166043_20130712_20170503_01_T1    LC81660432013193LGN01   
242  LC08_L1TP_165043_20130705_20170503_01_T1    LC81650432013186LGN01   
243  LC08_L1TP_166043_20130626_20170504_01_T1    LC81660432013177LGN02   
244  LC08_L1TP_165043_20130619_20170503_01_T1    LC81650432013170LGN01   
245  LC08_L1TP_166043_20130610_20170504_01_T1    LC81660432013161LGN01   
246  LC08_L1TP_165043_20130603_20170504_01_T1    LC81650432013154LGN01   
247  LC08_L1TP_166043_20130525_20170504_01_T1    LC81660432013145LGN01   
248  LC08_L1TP_165043_2

In [24]:
product = landsat8['Landsat Product Identifier']

In [28]:
for i in product.str.split('_'):
    print('Acquisition date:',i[3])

Acquisition date: 20180928
Acquisition date: 20180921
Acquisition date: 20180912
Acquisition date: 20180905
Acquisition date: 20180827
Acquisition date: 20180820
Acquisition date: 20180811
Acquisition date: 20180804
Acquisition date: 20180726
Acquisition date: 20180719
Acquisition date: 20180710
Acquisition date: 20180703
Acquisition date: 20180624
Acquisition date: 20180617
Acquisition date: 20180608
Acquisition date: 20180601
Acquisition date: 20180523
Acquisition date: 20180516
Acquisition date: 20180507
Acquisition date: 20180430
Acquisition date: 20180421
Acquisition date: 20180414
Acquisition date: 20180405
Acquisition date: 20180329
Acquisition date: 20180320
Acquisition date: 20180313
Acquisition date: 20180304
Acquisition date: 20180225
Acquisition date: 20180216
Acquisition date: 20180209
Acquisition date: 20180131
Acquisition date: 20180124
Acquisition date: 20180115
Acquisition date: 20180108
Acquisition date: 20171230
Acquisition date: 20171223
Acquisition date: 20171214
A

In [48]:
for i in landsat8['Scene Cloud Cover']:
    print(i)

0.0
0.0
0.0
3.82
0.0
0.0
0.0
0.23
0.0
0.1
0.0
0.0
0.0
0.0
0.0
0.0
0.0
20.29
0.32
0.0
0.0
0.0
42.06
0.01
1.12
0.14
0.14
5.71
0.35
0.77
1.85
1.52
0.3
0.8
0.39
2.54
0.43
0.7
28.37
34.73
0.41
4.47
0.0
0.04
0.0
0.0
0.0
0.0
0.29
0.0
0.47
0.03
0.0
0.02
0.0
0.01
2.07
0.0
0.0
0.0
0.0
0.0
27.6
2.34
1.27
15.53
19.5
0.0
5.7
15.08
49.7
4.07
29.29
12.28
29.03
39.29
26.74
81.04
0.3
0.25
0.47
0.59
1.3
0.07
100.0
2.83
0.24
0.17
0.0
0.15
0.0
0.03
0.0
0.0
7.82
0.0
3.29
0.31
0.0
0.01
0.09
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
41.8
0.0
1.04
0.0
0.01
21.91
0.04
56.39
73.61
23.49
0.42
0.13
0.31
38.13
0.29
9.81
0.56
95.29
7.67
0.21
0.06
6.44
0.27
0.69
0.07
84.47
1.06
0.0
0.0
0.0
0.0
0.0
0.26
-1.0
0.0
0.0
-1.0
1.14
0.0
-1.0
0.0
0.0
-1.0
0.0
1.56
-1.0
0.0
0.0
0.0
6.57
0.0
72.34
0.0
0.0
46.07
9.93
0.14
5.25
0.39
1.71
0.34
0.58
15.58
0.46
0.22
7.9
0.65
0.98
16.73
0.46
0.71
33.77
12.72
0.53
0.0
0.02
0.0
0.01
0.0
0.0
0.0
6.32
0.0
0.0
0.0
0.04
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.72
22.19
0.0
28.92
2.67
16.53
1.54
11.36


In [16]:
df['Acquisition Date']

0     2018-09-28
1     2018-09-21
2     2018-09-12
3     2018-09-05
4     2018-08-27
5     2018-08-20
6     2018-08-11
7     2018-08-04
8     2018-07-26
9     2018-07-19
10    2018-07-10
11    2018-07-03
12    2018-06-24
13    2018-06-17
14    2018-06-08
15    2018-06-01
16    2018-05-23
17    2018-05-16
18    2018-05-07
19    2018-04-30
20    2018-04-21
21    2018-04-14
22    2018-04-05
23    2018-03-29
24    2018-03-20
25    2018-03-13
26    2018-03-04
27    2018-02-25
28    2018-02-16
29    2018-02-09
         ...    
226   2013-11-26
227   2013-11-10
228   2013-11-01
229   2013-10-25
230   2013-10-16
231   2013-10-09
232   2013-09-23
233   2013-09-14
234   2013-09-07
235   2013-08-29
236   2013-08-22
237   2013-08-13
238   2013-08-06
239   2013-07-28
240   2013-07-21
241   2013-07-12
242   2013-07-05
243   2013-06-26
244   2013-06-19
245   2013-06-10
246   2013-06-03
247   2013-05-25
248   2013-05-18
249   2013-05-02
250   2013-04-23
251   2013-04-16
252   2013-04-09
253   2013-04-

In [5]:
# Finding all line that have B3 or B4
import re

land8_lst = []
with open('landsat8.txt', 'r') as file:
    for line in file:
        rex = re.compile(r'[B4|B3].TIF')
        if rex.search(line) != None:
            land8_lst.append(line)
        


In [12]:
land8_lst[0]

'gs://gcp-public-data-landsat/LC08/01/165/043/LC08_L1TP_165043_20130404_20170505_01_T1/LC08_L1TP_165043_20130404_20170505_01_T1_B3.TIF\n'

In [20]:
land8_tup = []
for line in land8_lst:
    lst = []
    spl = line.split("_")
    lst.append(spl[0].split("/")[-3])
    lst.append(spl[-5])
    land8_tup.append(lst)

In [23]:
land8_tup

[['165', '20130404'],
 ['165', '20130404'],
 ['165', '20130409'],
 ['165', '20130409'],
 ['165', '20130409'],
 ['165', '20130409'],
 ['165', '20130416'],
 ['165', '20130416'],
 ['165', '20130416'],
 ['165', '20130416'],
 ['165', '20130502'],
 ['165', '20130502'],
 ['165', '20130518'],
 ['165', '20130518'],
 ['165', '20130603'],
 ['165', '20130603'],
 ['165', '20130619'],
 ['165', '20130619'],
 ['165', '20130705'],
 ['165', '20130705'],
 ['165', '20130721'],
 ['165', '20130721'],
 ['165', '20130806'],
 ['165', '20130806'],
 ['165', '20130822'],
 ['165', '20130822'],
 ['165', '20130907'],
 ['165', '20130907'],
 ['165', '20130923'],
 ['165', '20130923'],
 ['165', '20131009'],
 ['165', '20131009'],
 ['165', '20131025'],
 ['165', '20131025'],
 ['165', '20131110'],
 ['165', '20131110'],
 ['165', '20131110'],
 ['165', '20131110'],
 ['165', '20131126'],
 ['165', '20131126'],
 ['165', '20131212'],
 ['165', '20131212'],
 ['165', '20131228'],
 ['165', '20131228'],
 ['165', '20140113'],
 ['165', '

In [37]:
landsat8['Acquisition Date'] = landsat8['Acquisition Date'].dt.strftime("%Y%m%d")

In [46]:
land8_nocloud = []
for i in landsat8['Acquisition Date']:
    for j in land8_tup:
        if str(i) == j[1]:
            print(i, j[1])


20180921 20180921
20180921 20180921
20180912 20180912
20180912 20180912
20180905 20180905
20180905 20180905
20180905 20180905
20180905 20180905
20180827 20180827
20180827 20180827
20180827 20180827
20180827 20180827
20180820 20180820
20180820 20180820
20180820 20180820
20180820 20180820
20180811 20180811
20180811 20180811
20180811 20180811
20180811 20180811
20180804 20180804
20180804 20180804
20180804 20180804
20180804 20180804
20180726 20180726
20180726 20180726
20180726 20180726
20180726 20180726
20180719 20180719
20180719 20180719
20180719 20180719
20180719 20180719
20180710 20180710
20180710 20180710
20180710 20180710
20180710 20180710
20180703 20180703
20180703 20180703
20180703 20180703
20180703 20180703
20180624 20180624
20180624 20180624
20180624 20180624
20180624 20180624
20180617 20180617
20180617 20180617
20180617 20180617
20180617 20180617
20180608 20180608
20180608 20180608
20180608 20180608
20180608 20180608
20180601 20180601
20180601 20180601
20180601 20180601
20180601 2

In [43]:
len(land8_nocloud)

666

In [44]:
land8_nocloud

['20180921',
 '20180921',
 '20180912',
 '20180912',
 '20180905',
 '20180905',
 '20180905',
 '20180905',
 '20180827',
 '20180827',
 '20180827',
 '20180827',
 '20180820',
 '20180820',
 '20180820',
 '20180820',
 '20180811',
 '20180811',
 '20180811',
 '20180811',
 '20180804',
 '20180804',
 '20180804',
 '20180804',
 '20180726',
 '20180726',
 '20180726',
 '20180726',
 '20180719',
 '20180719',
 '20180719',
 '20180719',
 '20180710',
 '20180710',
 '20180710',
 '20180710',
 '20180703',
 '20180703',
 '20180703',
 '20180703',
 '20180624',
 '20180624',
 '20180624',
 '20180624',
 '20180617',
 '20180617',
 '20180617',
 '20180617',
 '20180608',
 '20180608',
 '20180608',
 '20180608',
 '20180601',
 '20180601',
 '20180601',
 '20180601',
 '20180523',
 '20180523',
 '20180523',
 '20180523',
 '20180516',
 '20180516',
 '20180516',
 '20180516',
 '20180507',
 '20180507',
 '20180507',
 '20180507',
 '20180430',
 '20180430',
 '20180430',
 '20180430',
 '20180421',
 '20180421',
 '20180421',
 '20180421',
 '20180414',